In [33]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
import json
from dotenv import dotenv_values
from langchain.embeddings import OpenAIEmbeddings
from core.utils import load_chat_model, make_vector_store, tiktoken_len
from langchain_core.documents.base import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter 


In [34]:
env = dotenv_values("../.env")
os.environ["OPENAI_API_KEY"] = env["OPENAI_API_KEY"]

In [35]:
temp_docs = [
    "AMG 운전면허 모의 테스트는 LLM 어플리케이션 개발에 대한 제반지식을 위한 Toy project입니다.",
    "AMG는 뭔가요? 안장현 A, 명성은, M, 구윤택 G의 약자입니다.",
    "AMG 순서는 나이순서 입니다."
]
doc_list = []
for content in temp_docs:
    doc = Document(page_content=content)
    doc_list.append(doc)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                                length_function=tiktoken_len)#min_tokens=100, max_tokens=200)
texts = text_splitter.split_documents(doc_list)

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
rag = make_vector_store(docs=texts, llm_embedding=embeddings)


In [36]:
query = "AMG는 뭔가요?"
docs = rag.similarity_search_with_relevance_scores(query,k=2)
print(f"query: {query}")
for doc in docs:
    print(doc)
    


query: AMG는 뭔가요?
(Document(page_content='AMG는 뭔가요? 안장현 A, 명성은, M, 구윤택 G의 약자입니다.'), 0.8892099280185652)
(Document(page_content='AMG는 뭔가요? 안장현 A, 명성은, M, 구윤택 G의 약자입니다.'), 0.8892099280185652)


In [43]:

helpful_answer = f""" 
- 운전면허 모의고사를 풀어보세요.
- 문제가 주어 지면 답변은 객관식 정답과 해설로 해주세요.
- 답변 객관식 정답은 [ ] 안에 숫자로 답변 해주세요.
- 답변 해설은 < > 안에 해설을 한글로 답변 해주세요. 
- 답변 해설은 RAG를 넣어서 답변 해주세요.
- 당신은 RAG를 이용하여 객관식 번호와 해설을 답변 해야 합니다. 
- RAG 사용시 신뢰도가 0.75 낮다면 "학습되지 않는 내용 입니다" 이라고 답변 해주세요.
- 신뢰도가 0.75 높을 때만 객관식 번호와 해설을 답변 해주세요.
- 문제와 답변 예시를 보시면 이해가 빠르실 겁니다.
- 예시 : 문제: AMG는 뭔가요? 1. 안장현 A, 명성은, M, 구윤택 G의 약자입니다. < 안장현 A, 명성은, M, 구윤택 G의 약자입니다.>, 2. AMG는 메르세데스 에서 유래 했습니다.
3. AMG 순서는 나이순서 입니다. 4. AMG의 M은 몇살 인가요? 
RAG: AMG는 뭔가요? 안장현 A, 명성은, M, 구윤택 G의 약자입니다. 신뢰도: 0.76
답변: [1] 해설: < 안장현 A, 명성은, M, 구윤택 G의 약자입니다.>
\n\n
"""

In [44]:
chat_model = load_chat_model("gpt-3.5-turbo") #gpt-4-1106-preiew, gpt-3.5-turbo
question_list = temp_docs = [
    """
    문제: AMG 운전면허 모의 테스트 프로젝트의 목적은 무엇인가요? 
    1. 운전면허 시험 준비
    2. LLM 어플리케이션 개발에 대한 지식 증진
    3. 자동차 엔진 성능 테스트
    4. AMG 차량 판매 증진
    """,
    """
    문제:
    AMG 운전면허 모의 테스트 프로젝트는 어떤 종류의 프로젝트로 분류됩니까?

    1. 정부 프로젝트
    2. 학술 연구
    3. Toy project
    4. 상업적 벤처
    """,
    """
    문제: AMG의 이름 순서는 어떤 기준에 따라 정해졌나요?

    1. 직책 순서
    2. 나이 순서
    3. 알파벳 순서
    4. 입사 순서    
    """,
    """
    문제: OOP느 무엇의 약자인가요?
    
    1. 구글 메일 앱
    2. 구글 메일 어플리케이션
    3. 구글 메일 애플리케이션
    4. 구글 메일 어플
    """,
    """
    문제: 구글은 어떤 서비스를 제공하나요?
    
    1. 구글 메일 앱
    2. 구글 메일 어플리케이션
    3. 구글 메일 애플리케이션
    4. 구글 메일 어플
    """    
]
output_dict = {}
for idx, question in enumerate(question_list):
    docs = rag.similarity_search_with_relevance_scores(question,k=2) 
    content = docs[0][0].page_content
    simliarity = docs[0][1]   
    template = f"문제: {question}\n RAG: {content}, 신뢰도: {simliarity}" 
    input_question = f"{helpful_answer} {template}"
    result = chat_model.predict(input_question)
    output = f"{input_question}\n답변: {result}"
    print(f"question: {idx+1}--------------------------")
    print(output)
    print("\n\n")
    output_dict[idx+1] = {"question": question, "answer": result}
    

question: 1--------------------------
 
- 운전면허 모의고사를 풀어보세요.
- 문제가 주어 지면 답변은 객관식 정답과 해설로 해주세요.
- 답변 객관식 정답은 [ ] 안에 숫자로 답변 해주세요.
- 답변 해설은 < > 안에 해설을 한글로 답변 해주세요. 
- 답변 해설은 RAG를 넣어서 답변 해주세요.
- 당신은 RAG를 이용하여 객관식 번호와 해설을 답변 해야 합니다. 
- RAG 사용시 신뢰도가 0.75 낮다면 "학습되지 않는 내용 입니다" 이라고 답변 해주세요.
- 신뢰도가 0.75 높을 때만 객관식 번호와 해설을 답변 해주세요.
- 문제와 답변 예시를 보시면 이해가 빠르실 겁니다.
- 예시 : 문제: AMG는 뭔가요? 1. 안장현 A, 명성은, M, 구윤택 G의 약자입니다. < 안장현 A, 명성은, M, 구윤택 G의 약자입니다.>, 2. AMG는 메르세데스 에서 유래 했습니다.
3. AMG 순서는 나이순서 입니다. 4. AMG의 M은 몇살 인가요? 
RAG: AMG는 뭔가요? 안장현 A, 명성은, M, 구윤택 G의 약자입니다. 신뢰도: 0.76
답변: [1] 해설: < 안장현 A, 명성은, M, 구윤택 G의 약자입니다.>



 문제: 
    문제: AMG 운전면허 모의 테스트 프로젝트의 목적은 무엇인가요? 
    1. 운전면허 시험 준비
    2. LLM 어플리케이션 개발에 대한 지식 증진
    3. 자동차 엔진 성능 테스트
    4. AMG 차량 판매 증진
    
 RAG: AMG 운전면허 모의 테스트는 LLM 어플리케이션 개발에 대한 제반지식을 위한 Toy project입니다., 신뢰도: 0.8847654690861065
답변: 답변: [2] 해설: < LLM 어플리케이션 개발에 대한 지식 증진>



question: 2--------------------------
 
- 운전면허 모의고사를 풀어보세요.
- 문제가 주어 지면 답변은 객관식 정답과 해설로 해주세요.
- 답변 객관식 정답은 [ ] 안에 숫

In [45]:
with open("prmpt_tutorial_01.json", "w") as f:
    json.dump(output_dict, f, indent=4, ensure_ascii=False)